In [1]:
# Run in python console
import nltk; nltk.download('stopwords')
# Run in terminal or command prompt
!python3 -m spacy download en
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 1.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=8c45587ba3e89ffb5e9097469ac13c53c91417a7a8234b26e05a2eb0db7cd44f
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/titles.csv")
data['title']

0      истории философии понятия целом положительном ...
1      вопросу распространении теплоты текущей жидкос...
2       интегрировании посредством эллиптических функций
3      Наблюдение Казанской обсерватории метеорного п...
4               Летопись Физико-математического Общества
                             ...                        
480          Функция E(x) применение теории вероятностей
481    переходе кривых разных типов Пирсона гистограммам
482    отличительных свойствах сфер пространствах пос...
483                         Умножение приближенных чисел
484    Постоянные поля векторов тензоров порядка Рима...
Name: title, Length: 485, dtype: object

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['title']))
data_words

[['истории', 'философии', 'понятия', 'целом', 'положительном', 'числе'],
 ['вопросу',
  'распространении',
  'теплоты',
  'текущеи',
  'жидкости',
  'путем',
  'конвекции'],
 ['интегрировании', 'посредством', 'эллиптических', 'функции'],
 ['наблюдение', 'казанскои', 'обсерватории', 'метеорного', 'потока'],
 ['летопись', 'физико', 'математического', 'общества'],
 ['устав', 'физико', 'математического', 'общества'],
 ['список', 'членов', 'физико', 'математического', 'общества'],
 ['протоколы', 'заседании', 'физико', 'математического', 'общества'],
 ['список',
  'книг',
  'поступивших',
  'библиотеку',
  'физико',
  'математического',
  'общества'],
 ['отзыв'],
 ['теория', 'света'],
 ['поверхности'],
 ['обобщение', 'поверхностеи'],
 ['велечин'],
 ['протоколы', 'заседании', 'физико', 'математического', 'общества'],
 ['отзыв'],
 ['теория', 'света'],
 ['бернуллиевы', 'функции', 'произвольными', 'указателями'],
 ['заметка', 'числе', 'комбинации', 'гальванических', 'элементов'],
 ['протоколы', 

In [6]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words = remove_stopwords(data_words)
data_words

[['истории', 'философии', 'понятия', 'целом', 'положительном', 'числе'],
 ['вопросу',
  'распространении',
  'теплоты',
  'текущеи',
  'жидкости',
  'путем',
  'конвекции'],
 ['интегрировании', 'посредством', 'эллиптических', 'функции'],
 ['наблюдение', 'казанскои', 'обсерватории', 'метеорного', 'потока'],
 ['летопись', 'физико', 'математического', 'общества'],
 ['устав', 'физико', 'математического', 'общества'],
 ['список', 'членов', 'физико', 'математического', 'общества'],
 ['протоколы', 'заседании', 'физико', 'математического', 'общества'],
 ['список',
  'книг',
  'поступивших',
  'библиотеку',
  'физико',
  'математического',
  'общества'],
 ['отзыв'],
 ['теория', 'света'],
 ['поверхности'],
 ['обобщение', 'поверхностеи'],
 ['велечин'],
 ['протоколы', 'заседании', 'физико', 'математического', 'общества'],
 ['отзыв'],
 ['теория', 'света'],
 ['бернуллиевы', 'функции', 'произвольными', 'указателями'],
 ['заметка', 'числе', 'комбинации', 'гальванических', 'элементов'],
 ['протоколы', 

In [7]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1)],
 [(17, 1), (18, 1), (19, 1), (20, 1), (21, 1)],
 [(22, 1), (23, 1), (24, 1), (25, 1)],
 [(23, 1), (24, 1), (25, 1), (26, 1)],
 [(23, 1), (24, 1), (25, 1), (27, 1), (28, 1)],
 [(23, 1), (24, 1), (25, 1), (29, 1), (30, 1)],
 [(23, 1), (24, 1), (25, 1), (27, 1), (31, 1), (32, 1), (33, 1)],
 [(34, 1)],
 [(35, 1), (36, 1)],
 [(37, 1)],
 [(38, 1), (39, 1)],
 [(40, 1)],
 [(23, 1), (24, 1), (25, 1), (29, 1), (30, 1)],
 [(34, 1)],
 [(35, 1), (36, 1)],
 [(15, 1), (41, 1), (42, 1), (43, 1)],
 [(5, 1), (44, 1), (45, 1), (46, 1), (47, 1)],
 [(23, 1), (24, 1), (25, 1), (29, 1), (30, 1)],
 [(48, 1), (49, 1), (50, 1), (51, 1)],
 [(35, 1), (36, 1)],
 [(23, 1), (24, 1), (25, 1), (29, 1), (30, 1)],
 [(52, 1), (53, 1), (54, 1), (55, 1), (56, 2)],
 [(25, 1), (27, 1), (32, 1), (57, 1), (58, 1), (59, 1), (60, 1)],
 [(35, 2), (36, 1), (61, 1), (62, 1), (63, 1)

In [8]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('истории', 1),
  ('положительном', 1),
  ('понятия', 1),
  ('философии', 1),
  ('целом', 1),
  ('числе', 1)]]

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.print_topics()

[(0,
  '0.050*"лобачевского" + 0.038*"геометрии" + 0.035*"проф" + 0.020*"света" + 0.019*"отзыв" + 0.018*"системе" + 0.014*"линеичатои" + 0.013*"конкурсе" + 0.009*"разложении" + 0.009*"отзывы"'),
 (1,
  '0.047*"список" + 0.044*"членов" + 0.038*"теории" + 0.027*"приложение" + 0.016*"формул" + 0.016*"интегралов" + 0.015*"вероятностеи" + 0.015*"типа" + 0.013*"применение" + 0.013*"вывода"'),
 (2,
  '0.074*"теория" + 0.028*"научная" + 0.023*"хроника" + 0.020*"векторов" + 0.013*"способу" + 0.012*"проективная" + 0.012*"числе" + 0.009*"перевод" + 0.008*"академик" + 0.008*"сонин"'),
 (3,
  '0.019*"одном" + 0.016*"заметка" + 0.014*"определения" + 0.013*"несколько" + 0.013*"некоторых" + 0.013*"кривых" + 0.011*"слов" + 0.011*"mathematica" + 0.011*"rossica" + 0.011*"bibliographia"'),
 (4,
  '0.125*"отчет" + 0.097*"год" + 0.041*"секретаря" + 0.041*"библиотекаря" + 0.037*"казначея" + 0.034*"общества" + 0.031*"деятельности" + 0.023*"комиссии" + 0.022*"отчеты" + 0.013*"смета"'),
 (5,
  '0.182*"общества"

In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.308343  0.002932       1        1  16.378379
3      0.056733  0.115178       2        1  13.965601
6      0.063894 -0.249397       3        1  10.290545
7      0.059155  0.026505       4        1   9.893476
4     -0.110320 -0.025825       5        1   8.933377
9      0.032130  0.068684       6        1   8.924600
8      0.065306  0.032416       7        1   8.871514
0      0.018502  0.004099       8        1   8.711031
1      0.069803  0.008279       9        1   7.655404
2      0.053139  0.017129      10        1   6.376072, topic_info=                Term       Freq      Total Category  logprob  loglift
24          общества  74.000000  74.000000  Default  30.0000  30.0000
29         заседании  38.000000  38.000000  Default  29.0000  29.0000
76             отчет  25.000000  25.000000  Default  28.0000  28.0000
30         протоколы  34.000000  34.000000  Default  27.0000  27.0000
23   математического  33.000000  33.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
501          перевод   1.271561   2.989209  Topic10  -4.7173   1.8979
708          журнале   0.756390   2.108737  Topic10  -5.2368   1.7273
731           апреля   0.756390   2.053785  Topic10  -5.2368   1.7537
733         затмении   0.756390   2.053785  Topic10  -5.2368   1.7537
734        солнечном   0.756390   2.053785  Topic10  -5.2368   1.7537

[368 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
678      10  0.729720            avis
466       2  0.763950   bibliographia
789      10  0.729720       bruxelles
424       2  0.767817        concours
903       4  0.522158            dans
...     ...       ...             ...
955       9  0.568602  эллиптического
231       3  0.931873          эрмита
285       7  0.842275           этюды
822       3  0.756597         явлении
822       9  0.189149         явлении

[361 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 7, 8, 5, 10, 9, 1, 2, 3])